# Deep Reinforcement learning on PyTorch to play breakout

In [1]:
#import required packages
import random
import os
import sys
import time

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2


### Neural Network Architecture:

In [2]:


class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        
        self.total_actions = 3
        self.gamma = 0.99
        self.final_epsilon = 0.0001
        self.initial_epsilon = 0.1
        self.num_iterations = 2000000
        self.replay_mem_size = 10000
        self.batch_size = 32
        
        self.conv1 = nn.Conv2d(4, 32, 8, 4)
        self.relu1 = nn.ReLU(inplace = True)
        self.conv2 = nn.Conv2d(32, 64, 4, 2)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.relu3 = nn.ReLU(inplace = True)
        self.fc4 = nn.Linear(3136, 512)
        self.relu4 = nn.ReLU(inplace=True)
        self.fc5 = nn.Linear(512, self.total_actions)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        out = self.conv3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.fc5(out)
        
        return out

### Initialize weights:

In [3]:
def ini_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.uniform(m.weight, -0.01, 0.01)
        m.bias.data.fill_(0.01)

### Convert image to tensor:

In [4]:
def img_to_tensor(img):
    img_tensor = img.transpose(2,0,1)
    img_tensor = img_tensor.astype(np.float32)
    img_tensor = torch.from_numpy(img_tensor)
    #check if cuda is available
    if torch.cuda.is_available():
        img_tensor = img_tensor.cuda()
    return img_tensor

### Resizing and colour to grey scale conversion:

In [ ]:
def resize_and_rgb2grey(img):
    img = img[0:288, 0:404]
    img_data = cv2.cvtColor(cv2.resize(image, (84,84)), cv2.COLOR_BGR2GRAY)
    img_data[img_data > 0] = 255
    img_data = np.reshape(img_data, (84,84,1))
    return img_data

### Training Model: